In [1]:
import cv2
import argparse
import os
import glob
import yolo_detector as yolo
from sklearn import metrics

In [2]:
path = "/home/prom/work/other/opencv_study/rice_dnn"
weights = f"{path}/backup/yolov4-rice-split-dnsty_best.weights"
config = f"{path}/cfg/yolov4-rice-split-dnsty.cfg"
classes = f"{path}/obj.names"
images_path = f"{path}/dense_and_sparse_label"
confidence = 0.4
nms = 0.4

In [3]:
detector = yolo.YoloDetector(weights, config, classes)

In [4]:
# f"{path}/test.txt"
test_images = open(file=f"{path}/test.txt").readlines()
test_files = [f"{path}/{line[:-1]}" for line in test_images]
test_files[:4]
# !cat "{path}/test.txt"

['/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/IMG_20230115_164110.jpg',
 '/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/LINE_ALBUM_rice_100_dense_๒๓๐๒๑๖_0.jpg',
 '/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/LINE_ALBUM_rice_100_sparse_๒๓๐๒๑๖_23.jpg',
 '/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/LINE_ALBUM_rice_100_dense_๒๓๐๒๑๖_2.jpg']

In [5]:
all_files = []
exts = ['*.jpg', '*.png']
all_files = [f for ext in exts for f in glob.glob(os.path.join(images_path, ext))]
files = all_files
# files = test_files


predict = []
actual = []

In [6]:


for file in files:
    image = cv2.imread(file)

    # labels = detector.detect_and_label(image, args.confidence, args.nms)
    classes, scores, boxes = detector.detect(image, confidence, nms)
    labels = detector.label(image)
    predict.append(classes[0])

    # print(file)
    path = os.path.splitext(file)
    if os.path.isfile(path[0]+'.txt'):
        txtfile = open(path[0]+'.txt','r')
        lines = txtfile.readlines()
        for line in lines:
            classid = int(line.split(' ')[0])
            if len(classes) <=0 or classid != classes[0]:
                is_match = False
            actual.append(classid)
            break



In [7]:
# actual
print(metrics.confusion_matrix(actual,predict))



[[57  2  0  0  0  0  0  0]
 [ 1 56  1  0  0  0  0  0]
 [ 0  1 58  0  0  0  0  0]
 [ 0  0  0 58  0  0  0  0]
 [ 0  0  0  0  4  0  0  0]
 [ 0  0  0  0  0 28  2  0]
 [ 0  0  0  0  0  0 27  0]
 [ 0  0  0  0  0  0  0 30]]


In [8]:
# print(metrics.classification_report(actual, predict, target_names=detector.class_name, labels=detector.class_name))
print(metrics.classification_report(actual, predict))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97        59
           1       0.95      0.97      0.96        58
           2       0.98      0.98      0.98        59
           3       1.00      1.00      1.00        58
           4       1.00      1.00      1.00         4
           5       1.00      0.93      0.97        30
           6       0.93      1.00      0.96        27
           7       1.00      1.00      1.00        30

    accuracy                           0.98       325
   macro avg       0.98      0.98      0.98       325
weighted avg       0.98      0.98      0.98       325



In [9]:
!python3 rice_detector.py -c=/home/prom/work/other/opencv_study/rice_dnn/cfg/yolov4-rice-split-dnsty.cfg -w=/home/prom/work/other/opencv_study/rice_dnn/backup/yolov4-rice-split-dnsty_best.weights -cl=/home/prom/work/other/opencv_study/rice_dnn/obj.names -p=/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/LINE_ALBUM_rice_5_sparse_๒๓๐๒๑๖_27.jpg

/home/prom/work/other/opencv_study/rice_dnn/dense_and_sparse_label/LINE_ALBUM_rice_5_sparse_๒๓๐๒๑๖_27.jpg
Actual:  ['rice_5_sparse']  Match:  True
Predict:  ['rice_5_sparse: 0.9989']
